In [1]:
import numpy as np
from typing import List
import torch

from lib.TensorV2 import Tensor
from lib.NN import Layer

In [5]:
class Dense(Layer):
    def __init__(self, input_dim: int, output_dim: int):
        super().__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.weights = Tensor.randn(output_dim, input_dim) * 0.01 # init backwards (output_dim, input_dim) for computational efficiency
        self.biases = Tensor.zeros(output_dim)

    def forward(self, inputs: Tensor) -> Tensor:
        return inputs @ self.weights.T + self.biases # transpose weights for computational efficiency
    
    def parameters(self) -> List[Tensor]:
        return [self.weights, self.biases]

In [9]:
def test_dense():
    input_dim = 5
    output_dim = 3
    batch_size = 10

    pt_dense = torch.nn.Linear(input_dim, output_dim)
    dense = Dense(input_dim, output_dim)

    # copy the weights and biases from custom layer to pytorch layer
    with torch.no_grad():
        pt_dense.weight.copy_(torch.tensor(dense.weights.data))
        pt_dense.bias.copy_(torch.tensor(dense.biases.data))

    pt_input = torch.randn(batch_size, input_dim) # pytorch tensor
    input = Tensor(pt_input.numpy()) # copy the pytorch tensor

    pt_output = pt_dense(pt_input) # pytorch output
    output = dense(input) # custom output

    # compare the outputs
    assert np.allclose(pt_output.detach().numpy(), output.data, atol=1e-6)

In [10]:
test_dense()